In [13]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

usuario_db = os.getenv('POSTGRES_USER')
senha_db = os.getenv('POSTGRES_PASSWORD')
host_db = os.getenv('POSTGRES_HOST', 'localhost')
porta_db = os.getenv('POSTGRES_PORT', '5432')
nome_banco_db = os.getenv('POSTGRES_DB')

if not all([usuario_db, senha_db, nome_banco_db]):
    raise ValueError("Algumas variáveis de ambiente não foram encontradas no .env")

DATABASE_URL = f'postgresql://{usuario_db}:{senha_db}@{host_db}:{porta_db}/{nome_banco_db}'
engine = create_engine(DATABASE_URL)

print("Conexão criada.")

Conexão criada.


In [14]:
nome_schema_elt = 'staging'

with engine.begin() as connection:
    connection.execute(text(f"CREATE SCHEMA IF NOT EXISTS {nome_schema_elt};"))

print(f"Schema '{nome_schema_elt}' verificado.")

Schema 'staging' verificado.


In [15]:
arquivos_brutos = {
    'raw_servidores_2019': '../data/dados_servidores_2019.csv',
    'raw_servidores_2020': '../data/dados_servidores_2020.csv',
    'raw_servidores_2021': '../data/dados_servidores_2021.csv'
}

for nome_tabela, caminho_arquivo in arquivos_brutos.items():
    if not os.path.exists(caminho_arquivo):
        raise ValueError(f"Arquivo não encontrado: {caminho_arquivo}.")

In [16]:
print("Iniciando carga dos dados BRUTOS...")

for nome_tabela, caminho_arquivo in arquivos_brutos.items():
    try:
        print(f"Lendo o arquivo: {caminho_arquivo}")
        df_raw = pd.read_csv(
            caminho_arquivo,
            encoding='utf-8',
            on_bad_lines='skip',
            dtype=str
        )

        print(f"Carregando {len(df_raw)} linhas na tabela: {nome_tabela}...")

        df_raw.to_sql(
            nome_tabela,
            con=engine,
            if_exists='replace',
            index=False,
            schema=nome_schema_elt
        )
        print(f"Tabela {nome_tabela} carregada com sucesso.")

    except Exception as e:
        print(f"Erro ao processar {caminho_arquivo}: {e}")

print("\nProcesso de extração e carga concluído.")

Iniciando carga dos dados BRUTOS...
Lendo o arquivo: ../data/dados_servidores_2019.csv
Carregando 462500 linhas na tabela: raw_servidores_2019...
Tabela raw_servidores_2019 carregada com sucesso.
Lendo o arquivo: ../data/dados_servidores_2020.csv
Carregando 465209 linhas na tabela: raw_servidores_2020...
Tabela raw_servidores_2020 carregada com sucesso.
Lendo o arquivo: ../data/dados_servidores_2021.csv
Carregando 470623 linhas na tabela: raw_servidores_2021...
Tabela raw_servidores_2021 carregada com sucesso.

Processo de extração e carga concluído.
